In [1]:
# !python3 -m venv toxic-env
!source toxic-env/bin/activate

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
!pip install detoxify

  Obtaining dependency information for detoxify from https://files.pythonhosted.org/packages/1f/5c/94f765f08fe52473bc945284b7912032c099d0737cb8ad9235deb3b40d35/detoxify-0.5.2-py3-none-any.whl.metadata


In [3]:
from detoxify import Detoxify

# each model takes in either a string or a list of strings

results = Detoxify('original').predict(['I will kill you someday.','you are a soul sking creature'])

results

{'toxicity': [0.8530941605567932, 0.8805687427520752],
 'severe_toxicity': [0.07263469696044922, 0.002811698243021965],
 'obscene': [0.056023672223091125, 0.028627578169107437],
 'threat': [0.8342406153678894, 0.012782994657754898],
 'insult': [0.07935682684183121, 0.3828434348106384],
 'identity_attack': [0.03731388598680496, 0.006682781036943197]}

### Dataset 1: Toxic Comment

In [72]:
toxiccomment = pd.read_csv("..//Datasets/toxiccomment/toxiccomment.csv")
toxiccomment.head()

,id,comment_text,threat
0,55858b89f99e9bda,Hope he dies \n\nNow this Atheist filth's wife...,1
1,425a1dbdf740e9b8,"2006 (UTC)\n\n Removed Merge 17:15, 5 April",0
2,20c81b99f7adf557,John discuss it here \n\nSeems you don't like ...,0
3,af0dce6ce84974ec,"""\nTo answer your question, no. There is no si...",0
4,a069e6d6d1a2348d,"""\n But Arpad can cite any webpage he finds, o...",0


In [77]:
toxiccomment.to_csv('toxiccomment.csv',index=False)

In [26]:
toxiccomment_predictions = Detoxify('original').predict(toxiccomment['comment_text'].tolist())

In [30]:
toxic_unbiased_predictions = Detoxify('unbiased').predict(toxiccomment['comment_text'].tolist())

In [29]:
toxic_preds = pd.DataFrame(toxiccomment_predictions)
toxiccomment = pd.concat([toxiccomment,toxic_preds],axis=1)
toxiccomment.head()

,id,comment_text,threat,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,55858b89f99e9bda,Hope he dies \n\nNow this Atheist filth's wife...,1,0.986712,0.335980,0.447004,0.918513,0.740147,0.489629
1,425a1dbdf740e9b8,"2006 (UTC)\n\n Removed Merge 17:15, 5 April",0,0.000681,0.000113,0.000177,0.000119,0.000180,0.000138
2,20c81b99f7adf557,John discuss it here \n\nSeems you don't like ...,0,0.000605,0.000124,0.000180,0.000128,0.000171,0.000143
3,af0dce6ce84974ec,"""\nTo answer your question, no. There is no si...",0,0.000583,0.000123,0.000183,0.000124,0.000176,0.000138
4,a069e6d6d1a2348d,"""\n But Arpad can cite any webpage he finds, o...",0,0.000645,0.000118,0.000177,0.000119,0.000179,0.000140


In [66]:
# Add a new column 'pred_label' where 1 if 'threat' >= 0.7, otherwise 0
toxiccomment['pred_label'] = (toxiccomment['threat'] >= 0.7).astype(int)

# Display the resulting DataFrame
toxiccomment.head()

,id,comment_text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,true_label,pred_label
0,55858b89f99e9bda,Hope he dies \n\nNow this Atheist filth's wife...,0.986712,0.335980,0.447004,0.918513,0.740147,0.489629,1,1
1,425a1dbdf740e9b8,"2006 (UTC)\n\n Removed Merge 17:15, 5 April",0.000681,0.000113,0.000177,0.000119,0.000180,0.000138,0,0
2,20c81b99f7adf557,John discuss it here \n\nSeems you don't like ...,0.000605,0.000124,0.000180,0.000128,0.000171,0.000143,0,0
3,af0dce6ce84974ec,"""\nTo answer your question, no. There is no si...",0.000583,0.000123,0.000183,0.000124,0.000176,0.000138,0,0
4,a069e6d6d1a2348d,"""\n But Arpad can cite any webpage he finds, o...",0.000645,0.000118,0.000177,0.000119,0.000179,0.000140,0,0


In [67]:
detoxify_toxiccomment = toxiccomment

In [69]:
detoxify_toxiccomment.to_csv("detoxify_toxiccomment.csv",index=False)

In [74]:
toxiccomment.rename(columns={'threat':'true_label'},inplace=True)

In [75]:
unbiased_preds = pd.DataFrame(toxic_unbiased_predictions)
detoxify_unbiased_toxiccomment = pd.concat([toxiccomment,unbiased_preds],axis=1)
detoxify_unbiased_toxiccomment.head()

,id,comment_text,true_label,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,55858b89f99e9bda,Hope he dies \n\nNow this Atheist filth's wife...,1,0.976251,0.006515,0.064793,0.107087,0.504929,0.755125,0.003148
1,425a1dbdf740e9b8,"2006 (UTC)\n\n Removed Merge 17:15, 5 April",0,0.000536,0.000003,0.000056,0.000099,0.000071,0.000025,0.000023
2,20c81b99f7adf557,John discuss it here \n\nSeems you don't like ...,0,0.002967,0.000009,0.000249,0.000203,0.000120,0.000089,0.000130
3,af0dce6ce84974ec,"""\nTo answer your question, no. There is no si...",0,0.001307,0.000007,0.000141,0.000148,0.000077,0.000063,0.000097
4,a069e6d6d1a2348d,"""\n But Arpad can cite any webpage he finds, o...",0,0.000487,0.000002,0.000041,0.000078,0.000081,0.000019,0.000020


In [78]:
# Add a new column 'pred_label' where 1 if 'threat' >= 0.7, otherwise 0
detoxify_unbiased_toxiccomment['pred_label'] = (detoxify_unbiased_toxiccomment['threat'] >= 0.7).astype(int)

# Display the resulting DataFrame
detoxify_unbiased_toxiccomment.head()

,id,comment_text,true_label,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit,pred_label
0,55858b89f99e9bda,Hope he dies \n\nNow this Atheist filth's wife...,1,0.976251,0.006515,0.064793,0.107087,0.504929,0.755125,0.003148,1
1,425a1dbdf740e9b8,"2006 (UTC)\n\n Removed Merge 17:15, 5 April",0,0.000536,0.000003,0.000056,0.000099,0.000071,0.000025,0.000023,0
2,20c81b99f7adf557,John discuss it here \n\nSeems you don't like ...,0,0.002967,0.000009,0.000249,0.000203,0.000120,0.000089,0.000130,0
3,af0dce6ce84974ec,"""\nTo answer your question, no. There is no si...",0,0.001307,0.000007,0.000141,0.000148,0.000077,0.000063,0.000097,0
4,a069e6d6d1a2348d,"""\n But Arpad can cite any webpage he finds, o...",0,0.000487,0.000002,0.000041,0.000078,0.000081,0.000019,0.000020,0


In [79]:
detoxify_unbiased_toxiccomment.to_csv('detoxify_unbiased_toxiccomment.csv',index=False)

### Dataset 2: Jigsaw

In [6]:
jigsaw = pd.read_csv("jigsaw.csv")
jigsaw.head()

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,toxicity_annotator_count
0,627762,OH yes - Were those evil Christian Missionarie...,0.800000,0.000000,0.000000,0.000000,0.700000,0.500000,0.0,10
1,5137126,Blame men. There's always an excuse to blame ...,0.545455,0.000000,0.000000,0.000000,0.636364,0.272727,0.0,11
2,855753,And the woman exposing herself saying grab thi...,0.728571,0.000000,0.600000,0.628571,0.114286,0.300000,0.0,70
3,379144,You have no business making any comments on th...,0.594595,0.027027,0.040541,0.000000,0.121622,0.581081,0.0,74
4,641883,I guess the issue is people not willing to put...,0.764706,0.014706,0.117647,0.000000,0.000000,0.720588,0.0,68


In [7]:
jigsaw.rename(columns={'threat':'Threat_Jigsaw'},inplace=True)
jigsaw.head()

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,Threat_Jigsaw,toxicity_annotator_count
0,627762,OH yes - Were those evil Christian Missionarie...,0.800000,0.000000,0.000000,0.000000,0.700000,0.500000,0.0,10
1,5137126,Blame men. There's always an excuse to blame ...,0.545455,0.000000,0.000000,0.000000,0.636364,0.272727,0.0,11
2,855753,And the woman exposing herself saying grab thi...,0.728571,0.000000,0.600000,0.628571,0.114286,0.300000,0.0,70
3,379144,You have no business making any comments on th...,0.594595,0.027027,0.040541,0.000000,0.121622,0.581081,0.0,74
4,641883,I guess the issue is people not willing to put...,0.764706,0.014706,0.117647,0.000000,0.000000,0.720588,0.0,68


In [10]:
from tqdm import tqdm

# Copy the DataFrame to avoid modifying the original
detoxify_jigsaw = jigsaw.copy()

# Define batch size
batch_size = 128  # Adjust batch size based on your system's memory
predictions = []

# Initialize the Detoxify model
detoxify_model = Detoxify('original', device='cuda')

# Iterate through the dataset in batches with tqdm for progress tracking
for i in tqdm(range(0, len(jigsaw), batch_size), desc="Processing batches"):
    # Select a batch of comment_text
    batch_texts = jigsaw['comment_text'].tolist()[i:i + batch_size]
    
    # Predict on the batch
    batch_predictions = detoxify_model.predict(batch_texts)
    
    # Append the predictions to the results list
    predictions.append(batch_predictions)

# Combine all batch predictions into a single dictionary
jigsaw_predictions = {
    key: np.concatenate([batch[key] for batch in predictions], axis=0)
    for key in predictions[0].keys()
}

# Add predictions as columns to the original DataFrame
for label, values in jigsaw_predictions.items():
    detoxify_jigsaw[label] = values

# Display the updated DataFrame
detoxify_jigsaw.head()

Processing batches: 100%|███████████████████| 2030/2030 [21:41<00:00,  1.56it/s]


,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,Threat_Jigsaw,toxicity_annotator_count,threat
0,627762,OH yes - Were those evil Christian Missionarie...,0.034860,0.000580,0.001227,0.000000,0.009809,0.003477,0.0,10,0.000696
1,5137126,Blame men. There's always an excuse to blame ...,0.030388,0.000121,0.000582,0.000000,0.000539,0.000892,0.0,11,0.000230
2,855753,And the woman exposing herself saying grab thi...,0.926792,0.045506,0.794269,0.628571,0.208452,0.474446,0.0,70,0.007903
3,379144,You have no business making any comments on th...,0.895996,0.009237,0.228434,0.000000,0.166109,0.422160,0.0,74,0.007901
4,641883,I guess the issue is people not willing to put...,0.572338,0.000951,0.059534,0.000000,0.002842,0.080267,0.0,68,0.000775


In [12]:
detoxify_jigsaw.to_csv('detoxify_jigsaw.csv',index=False)

In [11]:
detoxify_unbiased_jigsaw = jigsaw.copy()
# Define batch size
batch_size = 64  # Adjust batch size based on your system's memory
unbiased_predictions = []

# Initialize the Detoxify model
unbiased_model = Detoxify('unbiased', device='cuda')

# Iterate through the dataset in batches with tqdm
for i in tqdm(range(0, len(detoxify_unbiased_jigsaw), batch_size), desc="Processing batches (unbiased model)"):
    # Select a batch of comment_text
    batch_texts = detoxify_unbiased_jigsaw['comment_text'].tolist()[i:i + batch_size]
    
    # Predict on the batch
    batch_predictions = unbiased_model.predict(batch_texts)
    
    # Append the predictions to the results list
    unbiased_predictions.append(batch_predictions)

# Combine all batch predictions into a single dictionary
jigsaw_unbiased_predictions = {
    key: np.concatenate([batch[key] for batch in unbiased_predictions], axis=0)
    for key in unbiased_predictions[0].keys()
}

# Add predictions as columns to the original DataFrame
for label, values in jigsaw_unbiased_predictions.items():
    detoxify_unbiased_jigsaw[f'unbiased_{label}'] = values

# Display the updated DataFrame
detoxify_unbiased_jigsaw.head()

Processing batches (unbiased model): 100%|██| 4059/4059 [20:24<00:00,  3.32it/s]


,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,Threat_Jigsaw,toxicity_annotator_count,unbiased_toxicity,unbiased_severe_toxicity,unbiased_obscene,unbiased_identity_attack,unbiased_insult,unbiased_threat,unbiased_sexual_explicit
0,627762,OH yes - Were those evil Christian Missionarie...,0.800000,0.000000,0.000000,0.000000,0.700000,0.500000,0.0,10,0.680323,0.000821,0.004664,0.647433,0.381809,0.004384,0.001478
1,5137126,Blame men. There's always an excuse to blame ...,0.545455,0.000000,0.000000,0.000000,0.636364,0.272727,0.0,11,0.135540,0.000167,0.000556,0.234763,0.014383,0.002641,0.004397
2,855753,And the woman exposing herself saying grab thi...,0.728571,0.000000,0.600000,0.628571,0.114286,0.300000,0.0,70,0.936026,0.040470,0.872100,0.024408,0.225219,0.005477,0.919514
3,379144,You have no business making any comments on th...,0.594595,0.027027,0.040541,0.000000,0.121622,0.581081,0.0,74,0.981255,0.000218,0.001696,0.039955,0.965053,0.000228,0.001291
4,641883,I guess the issue is people not willing to put...,0.764706,0.014706,0.117647,0.000000,0.000000,0.720588,0.0,68,0.985641,0.000128,0.003835,0.002144,0.977322,0.000212,0.000477


In [13]:
detoxify_unbiased_jigsaw.to_csv('detoxify_unbiased_jigsaw.csv',index=False)

### DATASET 3: Kaggle Suspicious

In [14]:
suspicious = pd.read_csv('..//kaggle_suspicious/suspicious_tweets.csv')
suspicious.head()

,message,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1
1,is upset that he can't update his Facebook by ...,1
2,@Kenichan I dived many times for the ball. Man...,1
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",1


In [16]:
detoxify_sus = suspicious.copy()

batch_size = 64  # Adjust batch size based on your system's memory
sus_predictions = []

# Initialize the Detoxify model
sus_detoxify = Detoxify('original', device='cuda')

# Iterate through the dataset in batches with tqdm
for i in tqdm(range(0, len(suspicious), batch_size), desc="Processing batches (Original model)"):
    # Select a batch of comment_text
    batch_texts = suspicious['message'].tolist()[i:i + batch_size]
    
    # Predict on the batch
    batch_predictions = sus_detoxify.predict(batch_texts)
    
    # Append the predictions to the results list
    sus_predictions.append(batch_predictions)

# Combine all batch predictions into a single dictionary
detoxify_sus_preds = {
    key: np.concatenate([batch[key] for batch in sus_predictions], axis=0)
    for key in sus_predictions[0].keys()
}

# Add predictions as columns to the original DataFrame
for label, values in detoxify_sus_preds.items():
    detoxify_sus[f'{label}'] = values

# Display the updated DataFrame
detoxify_sus.head()

Processing batches (Original model): 100%|████| 938/938 [01:06<00:00, 14.08it/s]


,message,label,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1,0.006961,0.000096,0.000533,0.000129,0.000443,0.000178
1,is upset that he can't update his Facebook by ...,1,0.201620,0.000253,0.003418,0.000716,0.003902,0.000669
2,@Kenichan I dived many times for the ball. Man...,1,0.001220,0.000098,0.000187,0.000108,0.000201,0.000145
3,my whole body feels itchy and like its on fire,0,0.251167,0.001026,0.017238,0.001236,0.001639,0.000919
4,"@nationwideclass no, it's not behaving at all....",1,0.003601,0.000092,0.000269,0.000116,0.000265,0.000164


In [17]:
detoxify_sus.to_csv('detoxify_sus.csv',index=False)

In [18]:
detoxify_unbiased_sus = suspicious.copy()

batch_size = 128  # Adjust batch size based on your system's memory
sus_predictions = []

# Initialize the Detoxify model
sus_detoxify = Detoxify('unbiased', device='cuda')

# Iterate through the dataset in batches with tqdm
for i in tqdm(range(0, len(suspicious), batch_size), desc="Processing batches (Original model)"):
    # Select a batch of comment_text
    batch_texts = suspicious['message'].tolist()[i:i + batch_size]
    
    # Predict on the batch
    batch_predictions = sus_detoxify.predict(batch_texts)
    
    # Append the predictions to the results list
    sus_predictions.append(batch_predictions)

# Combine all batch predictions into a single dictionary
detoxify_unbiased_sus_preds = {
    key: np.concatenate([batch[key] for batch in sus_predictions], axis=0)
    for key in sus_predictions[0].keys()
}

# Add predictions as columns to the original DataFrame
for label, values in detoxify_unbiased_sus_preds.items():
    detoxify_unbiased_sus[f'{label}'] = values

# Display the updated DataFrame
detoxify_unbiased_sus.head()

Processing batches (Original model): 100%|████| 469/469 [01:00<00:00,  7.75it/s]


,message,label,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1,0.005108,0.000003,0.000394,0.000163,0.001283,0.000056,0.000058
1,is upset that he can't update his Facebook by ...,1,0.002543,0.000002,0.000105,0.000180,0.000497,0.000068,0.000044
2,@Kenichan I dived many times for the ball. Man...,1,0.001766,0.000003,0.000122,0.000121,0.000351,0.000056,0.000095
3,my whole body feels itchy and like its on fire,0,0.023170,0.000013,0.001084,0.000696,0.002748,0.000658,0.000633
4,"@nationwideclass no, it's not behaving at all....",1,0.039335,0.000010,0.001244,0.001034,0.005948,0.000704,0.000148


In [19]:
detoxify_unbiased_sus.to_csv('detoxify_unbiased_suspicious.csv',index=False)

### Dataset 4: Combined Toxicity Profanity V2

In [26]:
combined = pd.read_csv('..//Combined_Toxicity_Profanity/combined_en.csv')
combined.head()

,text,labels,encoded_labels
0,Go ahead and merge what you think is necessary...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"""\n\n Merging Red Hand of Doom and Reliability...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,You never responded to this: any luck? —,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"\fuck you, who the shit is ari?",['toxic' 'profane' 'insult'],"[1, 1, 1, 0, 0, 0, 0, 0, 0]"
4,"This is a school computer, so any edits which ...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [22]:
# Create a copy of the DataFrame
combined_copy = combined.copy()

# Define batch size
batch_size = 128  # Adjust batch size based on your system's memory
predictions = []

# Initialize the Detoxify model
detoxify_model = Detoxify('original', device='cuda')

# Process the DataFrame in batches
for i in tqdm(range(0, len(combined_copy), batch_size), desc="Processing batches"):
    # Select a batch of text
    batch_texts = combined_copy['text'].tolist()[i:i + batch_size]
    
    # Predict on the batch
    batch_predictions = detoxify_model.predict(batch_texts)
    
    # Append the batch predictions to the results list
    predictions.append(batch_predictions)

# Combine all batch predictions into a single dictionary
combined_predictions = {
    key: np.concatenate([batch[key] for batch in predictions], axis=0)
    for key in predictions[0].keys()
}

# Add predictions as new columns to the copied DataFrame
for label, values in combined_predictions.items():
    combined_copy[label] = values

# Display the updated DataFrame
combined_copy.head()

Processing batches: 100%|███████████████████| 2215/2215 [27:59<00:00,  1.32it/s]


,text,labels,encoded_labels,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,Go ahead and merge what you think is necessary...,[],[0 0 0 0 0 0 0 0 0],0.000634,0.000122,0.000174,0.000138,0.000182,0.000142
1,"""\n\n Merging Red Hand of Doom and Reliability...",[],[0 0 0 0 0 0 0 0 0],0.000591,0.000128,0.000189,0.000132,0.000176,0.000146
2,You never responded to this: any luck? —,[],[0 0 0 0 0 0 0 0 0],0.000828,0.000107,0.000179,0.000119,0.000190,0.000134
3,"\fuck you, who the shit is ari?",['toxic' 'profane' 'insult'],[1 1 1 0 0 0 0 0 0],0.997464,0.325149,0.989654,0.002681,0.914595,0.010613
4,"This is a school computer, so any edits which ...",[],[0 0 0 0 0 0 0 0 0],0.000705,0.000114,0.000175,0.000121,0.000178,0.000136


In [25]:
combined_copy.to_csv('detoxify_combined_toxic_profane.csv',index=False)

In [27]:
# Create a copy of the DataFrame
combined_unbiased_copy = combined.copy()

# Define batch size
batch_size = 128  # Adjust batch size based on your system's memory
predictions = []

# Initialize the Detoxify model
detoxify_model = Detoxify('unbiased', device='cuda')

# Process the DataFrame in batches
for i in tqdm(range(0, len(combined_unbiased_copy), batch_size), desc="Processing batches"):
    # Select a batch of text
    batch_texts = combined_unbiased_copy['text'].tolist()[i:i + batch_size]
    
    # Predict on the batch
    batch_predictions = detoxify_model.predict(batch_texts)
    
    # Append the batch predictions to the results list
    predictions.append(batch_predictions)

# Combine all batch predictions into a single dictionary
combined_unbiased_predictions = {
    key: np.concatenate([batch[key] for batch in predictions], axis=0)
    for key in predictions[0].keys()
}

# Add predictions as new columns to the copied DataFrame
for label, values in combined_unbiased_predictions.items():
    combined_unbiased_copy[label] = values

# Display the updated DataFrame
combined_unbiased_copy.head()

Processing batches: 100%|███████████████████| 2215/2215 [25:59<00:00,  1.42it/s]


,text,labels,encoded_labels,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,Go ahead and merge what you think is necessary...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.013276,0.000020,0.001029,0.000361,0.000272,0.000346,0.000402
1,"""\n\n Merging Red Hand of Doom and Reliability...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.000925,0.000005,0.000155,0.000120,0.000093,0.000031,0.000058
2,You never responded to this: any luck? —,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.001012,0.000002,0.000069,0.000081,0.000219,0.000031,0.000028
3,"\fuck you, who the shit is ari?",['toxic' 'profane' 'insult'],"[1, 1, 1, 0, 0, 0, 0, 0, 0]",0.995545,0.166072,0.983968,0.033663,0.811409,0.003488,0.057676
4,"This is a school computer, so any edits which ...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.001679,0.000005,0.000107,0.000195,0.000096,0.000059,0.000076


In [28]:
combined_unbiased_copy.to_csv('Detoxify_unbiased_combined_toxic_profane.csv',index=False)